In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [2]:
#### 본문to형태소(morphs) 파일
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_6월_형태소분석.txt", encoding='UTF-8')
df['morphs']=df['morphs'].apply(lambda x : x.strip("'][").split("', '"))
print(df.shape, df.columns)
docs = df['morphs'].to_list()
print(len(docs))

(548091, 4) Index(['split_str', 'org_idx', 'morphs', 'pos_tag'], dtype='object')
548091


In [3]:
df

,split_str,org_idx,morphs,pos_tag
0,7월 개봉 예정 영화 신작 영화추천 신작 영화 추천 극장 개봉 예정 영화 7월 리부...,0,"[7월, 개봉, 예정, 영화, 신작, 영화, 추천, 신작, 영화, 추천, 극장, 개...","[('7월', 'Number'), ('개봉', 'Noun'), ('예정', 'Nou..."
1,문제는 쥬라기 월드 탑건 신비한 동물사전 레전드 영화의 리부트가 연일 관객들의 기대...,0,"[문제, 는, 쥬라기, 월드, 탑건, 신비한, 동물, 사전, 레전드, 영화, 의, ...","[('문제', 'Noun'), ('는', 'Josa'), ('쥬라기', 'Noun'..."
2,어린 시절의 향수를 진하게 불러주는 오마주는 반갑지만 그때 느꼈던 감동과 재미는 다...,0,"[어린, 시절, 의, 향수, 를, 진하게, 불러주는, 오마주, 는, 반갑지만, 그때...","[('어리다', 'Verb'), ('시절', 'Noun'), ('의', 'Josa'..."
3,저야 극장에서 보는 어지간한 영화는 재밌게 보고 나오는 편입니다만 그래서 극장 관람...,0,"[저, 야, 극장, 에서, 보는, 어지간한, 영화, 는, 재밌게, 보고, 나오는, ...","[('저', 'Noun'), ('야', 'Josa'), ('극장', 'Noun'),..."
4,그래서 2차 3차 가공해서 영화를 해석하고 추천하는 글을 쓰는 데는 생각이 참 많아...,0,"[그래서, 2, 차, 3, 차, 가공, 해서, 영화, 를, 해석, 하고, 추천, 하...","[('그래서', 'Adverb'), ('2', 'Number'), ('차', 'No..."
...,...,...,...,...
548086,이런 의문들은 또다시 석기를 그 지역의 환경적 맥락에서 떼어 놓고 보기 때문에 생겨...,11966,"[이런, 의문, 들, 은, 또다시, 석기, 를, 그, 지역, 의, 환경, 적, 맥락...","[('이렇다', 'Adjective'), ('의문', 'Noun'), ('들', '..."
548087,어떤 종이든 간에 대나무가 사방에 깔린 열대우림 지역에서는 대나무를 이용하는 것이 ...,11966,"[어떤, 종이, 든, 간, 에, 대나무, 가, 사방, 에, 깔린, 열대우림, 지역,...","[('어떻다', 'Adjective'), ('종이', 'Noun'), ('든', '..."
548088,그렇다면 왜 3만 년 전부터 도구가 변하기 시작하는 것일까 만약 도구를 만든 사람 ...,11966,"[그렇다면, 왜, 3만, 년, 전, 부터, 도구, 가, 변하기, 시작, 하는, 것,...","[('그렇다', 'Adjective'), ('왜', 'Noun'), ('3만', '..."
548089,바로 이때부터 진짜로 정교한 현대 호모 사피엔스의 도구가 발명되기 시작한다,11966,"[바로, 이, 때, 부터, 진짜, 로, 정교한, 현대, 호모, 사피, 엔, 스, 의...","[('바로', 'Noun'), ('이', 'Determiner'), ('때', 'N..."


In [4]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
# for i in tqdm(range(len(docs))):
#     docs2[i] = docs2[i].split()

In [5]:
#### 명사 빈도 상위 50개 단어 파일
topwords_df = pd.read_csv("../dataset/2022/본문_형태소분석/6월/나무문화_6월_형태소분석(명사).csv", encoding='UTF-8')
print(topwords_df.shape, topwords_df.columns)
topwords50 = topwords_df['명사'].to_list()[:50]
print(len(topwords50))
print(topwords50)

(119334, 2) Index(['명사', '명사빈도'], dtype='object')
50
['것', '수', '그', '이', '곳', '사람', '때', '등', '우리', '말', '더', '생각', '내', '나', '문화', '마을', '시간', '나무', '위', '길', '사진', '전', '안', '중', '제', '의', '일', '때문', '층', '여행', '차', '날', '하나', '아이', '단지', '공간', '집', '위해', '시작', '개', '공원', '카페', '작품', '거', '저', '다른', '마음', '한국', '책', '또']


In [6]:
#### 중복 단어 제거하여 리스트 생성
unique_lines = [list(set(line)) for line in tqdm(docs2)]

100%|██████████| 548091/548091 [00:04<00:00, 128143.81it/s]


In [7]:
#### 중복 제거 잘 됐나 개수로 확인
print(len(docs[0]))
print(len(unique_lines[0]))

27
17


In [8]:
#### 전체 단어쌍 빈도 dict 형식으로 생성
freq_count = {}   #동시출현 빈도가 저장될 dict
for words in tqdm(unique_lines):
    # 전체 단어에 대한 동시 출현 빈도
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue
            elif a>b:
                freq_count[b, a] = freq_count.get((b, a),0) + 1
            else :
                freq_count[a, b] = freq_count.get((a, b),0) + 1

100%|██████████| 548091/548091 [06:15<00:00, 1458.33it/s]


In [9]:
#### 딕셔너리를 데이터프레임에 담기
tt_freq_df=pd.DataFrame.from_dict(freq_count, orient='index')

In [10]:
#### 딕트to데이터프레임 저장
tt_freq_df.to_csv("../dataset/2022/단어동시출현행렬/6월/단어동시출현_딕셔너리_6월.csv", encoding='UTF-8')

In [11]:
#### dict형식을 컬럼명 지정해서 데이터프레임으로 만들기
list1 = [(tt_freq_df.index[i][0], tt_freq_df.index[i][1], tt_freq_df[0][i]) for i in tqdm(range(len(tt_freq_df)))]
freq_df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
print(freq_df2.shape)
freq_df2.head()

100%|██████████| 102776335/102776335 [12:02<00:00, 142162.98it/s]


(102776335, 3)


,term1,term2,freq
0,거대한,한자리,1
1,거대한,크게,12
2,거대한,영화,15
3,가,거대한,506
4,거대한,상업,2


In [12]:
#### 컬럼명 지정한 dict_to_데이터프레임 저장
freq_df2.to_csv("../dataset/2022/단어동시출현행렬/6월/단어동시출현_df_6월.csv", index=False, encoding='UTF-8')

In [13]:
#### 50위 이내에 속한 단어들로 이루어진 Dataframe을 추출
list2 =[(freq_df2.loc[i][0], freq_df2.loc[i][1], freq_df2.loc[i][2]) for i in tqdm(range(len(freq_df2))) if (freq_df2['term1'][i] in topwords50) &(freq_df2['term2'][i] in topwords50)]
freq_df3 = pd.DataFrame(list2, columns=["term1","term2","freq"])
freq_df3

100%|██████████| 102776335/102776335 [21:07<00:00, 81079.25it/s]


,term1,term2,freq
0,의,이,111137
1,등,마음,459
2,등,의,16102
3,다른,등,895
4,더,등,986
...,...,...,...
1220,단지,위,182
1221,공원,책,71
1222,단지,시작,86
1223,단지,때문,104


In [14]:
#### 상위50위 단어들로 이루어진 단어쌍 데이터프레임 저장
freq_df3.to_csv("../dataset/2022/단어동시출현행렬/6월/단어동시출현_df_상위50_6월.csv", index=False, encoding='UTF-8')

In [15]:
#### 공출현빈도(1-mode matrix) 만들기
word_co_matx = np.zeros((50,50))
mat_idx = topwords50 # 단어동시출현행렬이니까 인덱스와 칼럼을 동일하게 지정
mat_col = topwords50
word_co_matx = pd.DataFrame(word_co_matx, index=mat_idx, columns=mat_col)
print(word_co_matx.shape)


#### 단어쌍의 첫 번째 단어가 인덱스와 같다 and 두 번째 단어가 칼럼과 같다 → 단어쌍빈도를 해당 셀 값에 삽입
for n in tqdm(range(len(freq_df3))):
    for idx in mat_idx:
        if idx == freq_df3['term1'][n]:
            for col in mat_col:
                if col == freq_df3['term2'][n]:
                    word_co_matx.loc[idx, col] = freq_df3['freq'][n]
                    word_co_matx.loc[col, idx] = freq_df3['freq'][n]

(50, 50)


100%|██████████| 1225/1225 [00:00<00:00, 1534.15it/s]


In [18]:
#### 1-mode 매트릭스 확인하기
word_co_matx

,것,수,그,이,곳,사람,때,등,우리,말,더,생각,내,나,문화,마을,시간,나무,위,길,사진,전,안,중,제,의,일,때문,층,여행,차,날,하나,아이,단지,공간,집,위해,시작,개,공원,카페,작품,거,저,다른,마음,한국,책,또
것,0.0,15547.0,15542.0,58682.0,5176.0,8647.0,7838.0,4623.0,7065.0,7226.0,6088.0,6855.0,6513.0,7905.0,3099.0,1140.0,3688.0,2941.0,4326.0,2327.0,2243.0,3855.0,4566.0,4402.0,2856.0,43389.0,4920.0,4794.0,1072.0,1694.0,1216.0,2496.0,3402.0,2336.0,542.0,1964.0,2173.0,3345.0,2604.0,1969.0,971.0,1098.0,1996.0,2284.0,3178.0,3796.0,3137.0,1784.0,2198.0,2633.0
수,15547.0,0.0,8779.0,38143.0,7008.0,5616.0,4843.0,5998.0,4446.0,4059.0,4535.0,3983.0,4375.0,5823.0,3822.0,1850.0,3945.0,2975.0,3286.0,2760.0,2490.0,3231.0,3160.0,3198.0,2772.0,33068.0,3528.0,3480.0,1685.0,2006.0,1438.0,1764.0,2336.0,2422.0,837.0,3345.0,1690.0,2278.0,1758.0,2067.0,2181.0,1647.0,2164.0,1618.0,1956.0,2535.0,2227.0,1536.0,1749.0,1897.0
그,15542.0,8779.0,0.0,33199.0,4587.0,6197.0,5541.0,3117.0,4126.0,4672.0,3237.0,3471.0,4041.0,5081.0,2028.0,1055.0,2077.0,2027.0,3005.0,1767.0,1383.0,2894.0,3021.0,3689.0,2067.0,31646.0,3133.0,2721.0,661.0,874.0,920.0,1978.0,2165.0,1304.0,507.0,1188.0,1649.0,2235.0,1783.0,1293.0,655.0,691.0,1655.0,1521.0,1656.0,2148.0,2017.0,1109.0,1390.0,1819.0
이,58682.0,38143.0,33199.0,0.0,24449.0,22624.0,19739.0,16234.0,14386.0,14761.0,13967.0,14735.0,15064.0,19225.0,9844.0,5736.0,11745.0,10093.0,10374.0,8543.0,8874.0,11584.0,12517.0,11446.0,9199.0,111137.0,10595.0,10324.0,4686.0,5448.0,4365.0,9432.0,7877.0,7316.0,2067.0,7644.0,7225.0,7543.0,7589.0,6823.0,4998.0,5235.0,6412.0,7518.0,8734.0,8062.0,7891.0,5483.0,6059.0,6871.0
곳,5176.0,7008.0,4587.0,24449.0,0.0,2543.0,2457.0,2352.0,1594.0,1195.0,1880.0,1789.0,1947.0,2511.0,2226.0,1724.0,2012.0,1784.0,1328.0,2207.0,1963.0,1696.0,1700.0,2070.0,1519.0,14219.0,1162.0,1076.0,899.0,2377.0,710.0,1125.0,1195.0,1781.0,333.0,1765.0,1210.0,1083.0,950.0,1083.0,1833.0,1646.0,595.0,775.0,1477.0,1417.0,1065.0,762.0,582.0,1024.0
사람,8647.0,5616.0,6197.0,22624.0,2543.0,0.0,2935.0,1469.0,2392.0,2897.0,2127.0,2718.0,2668.0,3780.0,1283.0,830.0,1466.0,1149.0,1507.0,1221.0,1170.0,1480.0,2078.0,1666.0,1002.0,15423.0,2139.0,1597.0,373.0,735.0,473.0,1095.0,1187.0,688.0,143.0,754.0,1020.0,1310.0,922.0,782.0,418.0,569.0,674.0,1131.0,1235.0,1981.0,1483.0,683.0,875.0,1075.0
때,7838.0,4843.0,5541.0,19739.0,2457.0,2935.0,0.0,1615.0,2375.0,2261.0,2063.0,2256.0,2588.0,3334.0,967.0,488.0,1554.0,1212.0,1317.0,1051.0,1148.0,1737.0,2045.0,1585.0,1340.0,13447.0,1773.0,1322.0,450.0,903.0,554.0,1322.0,1156.0,966.0,157.0,597.0,1185.0,1083.0,1271.0,748.0,424.0,498.0,555.0,1245.0,1282.0,1171.0,1149.0,656.0,696.0,1036.0
등,4623.0,5998.0,3117.0,16234.0,2352.0,1469.0,1615.0,0.0,1010.0,1216.0,986.0,796.0,1536.0,2395.0,2450.0,881.0,1152.0,1580.0,1914.0,910.0,802.0,1820.0,1001.0,1775.0,1738.0,16102.0,1226.0,819.0,748.0,761.0,599.0,596.0,697.0,672.0,563.0,1351.0,681.0,1231.0,927.0,1447.0,1017.0,643.0,1005.0,319.0,546.0,895.0,459.0,1158.0,720.0,533.0
우리,7065.0,4446.0,4126.0,14386.0,1594.0,2392.0,2375.0,1010.0,0.0,1927.0,1674.0,1983.0,1730.0,2212.0,1470.0,577.0,1357.0,959.0,1222.0,954.0,827.0,1198.0,1582.0,1121.0,762.0,13116.0,1513.0,1152.0,337.0,587.0,534.0,949.0,965.0,1012.0,165.0,502.0,1292.0,1084.0,724.0,557.0,333.0,465.0,529.0,960.0,819.0,925.0,1184.0,627.0,738.0,955.0
말,7226.0,4059.0,4672.0,14761.0,1195.0,2897.0,2261.0,1216.0,1927.0,0.0,1445.0,1763.0,2262.0,2603.0,790.0,328.0,844.0,768.0,1130.0,593.0,500.0,1099.0,1638.0,999.0,881.0,11835.0,1498.0,1070.0,199.0,375.0,333.0,727.0,878.0,641.0,167.0,316.0,598.0,846.0,674.0,552.0,218.0,235.0,390.0,889.0,846.0,967.0,1048.0,500.0,570.0,775.0


In [17]:
#### 공출현빈도 데이터프레임 저장하기~~
#### ※ 인덱스가 저장돼야 하므로 index 파라미터 지정하지 말 것.
word_co_matx.to_csv("../dataset/2022/단어동시출현행렬/6월/공출현빈도_df_6월.csv", encoding='UTF-8')